In [ ]:
!pip install -q transformers accelerate bitsandbytes fastapi uvicorn nest_asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login #get auth token from your huggingface account
login()

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn
import threading

# -----------------------
# FastAPI App
# -----------------------
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# -----------------------
# Model Setup
# -----------------------
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map={"": 0},   # FORCE full GPU
)

model.eval()

print("Model loaded successfully!")

# -----------------------
# Request Schema
# -----------------------
class PromptRequest(BaseModel):
    prompt: str
    max_tokens: int = 150

# -----------------------
# Health Check
# -----------------------
@app.get("/")
def health():
    return {"status": "Mistral 7B Quantized API running"}

# -----------------------
# Generation Endpoint
# -----------------------
@app.post("/generate")
def generate_text(request: PromptRequest):

    inputs = tokenizer(
        request.prompt,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=request.max_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {"response": response}

# -----------------------
# Run Server
# -----------------------
nest_asyncio.apply()

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()


Loading tokenizer...
Loading model...


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded successfully!


In [ ]:
from pyngrok import ngrok
#get auth token from ngrok, get new one everytime run time disconnects
ngrok.set_auth_token("39gRglSkUC49XSZSdczmCmDn9Jr_2PkDZJWxYRuZhy8soYvx")

public_url = ngrok.connect(8000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://reiko-pseudolegislative-heteronymously.ngrok-free.dev" -> "http://localhost:8000"
